##INGEST

In [ ]:
import numpy as np
import pandas as pd
import string

In [ ]:
df=pd.read_json('News_Category_Dataset_v2.json',lines=True)

##PREPROCESSING

In [ ]:
lis=[ 'WORLD NEWS',
       'SPORTS', 'MONEY',
       'WOMEN','SCIENCE']

In [ ]:
df=df.loc[df['category'].isin(lis)]
df=df.reset_index(drop=True)

In [ ]:
df

,category,headline,authors,link,short_description,date
0,WORLD NEWS,South Korean President Meets North Korea's Kim...,,https://www.huffingtonpost.com/entry/south-kor...,The two met to pave the way for a summit betwe...,2018-05-26
1,WORLD NEWS,North Korea Still Open To Talks After Trump Ca...,"Josh Smith and Christine Kim, Reuters",https://www.huffingtonpost.com/entry/north-kor...,Trump’s announcement came after repeated threa...,2018-05-25
2,WORLD NEWS,2 Men Detonate Bomb Inside Indian Restaurant N...,,https://www.huffingtonpost.com/entry/mississau...,"Fifteen people were taken to the hospital, thr...",2018-05-25
3,WORLD NEWS,Thousands Travel Home To Ireland To Vote On Ab...,Antonia Blumberg,https://www.huffingtonpost.com/entry/irish-tra...,Just try to read these #HomeToVote tweets with...,2018-05-25
4,WORLD NEWS,Irish Voters Set To Liberalize Abortion Laws I...,,https://www.huffingtonpost.com/entry/ireland-a...,Vote counting will begin Saturday.,2018-05-25
...,...,...,...,...,...,...
14431,SCIENCE,Aurora Borealis Caused By Huge Solar Storm Shi...,,https://www.huffingtonpost.com/entry/aurora-fr...,Aurora borealis can typically only be seen at ...,2012-01-28
14432,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
14433,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
14434,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


In [ ]:
df["text"] = df["headline"] + df["short_description"] 

In [ ]:
lis=[ 'WORLD NEWS',
       'SPORTS', 'MONEY',
       'WOMEN','SCIENCE']

In [ ]:
df=df.loc[df['category'].isin(lis)]
df=df.reset_index(drop=True)

In [ ]:
df["text"] = df["headline"] + df["short_description"] 

In [ ]:

df['text'][1]


'North Korea Still Open To Talks After Trump Cancels SummitTrump’s announcement came after repeated threats by North Korea to pull out of the summit over what it saw as confrontational remarks by U.S. officials.'

In [ ]:
def removePunct(txt):
  txt_nopunct="".join([c for c in txt if c not in string.punctuation])
  return txt_nopunct

In [ ]:
df['textClean']=df['text'].apply(lambda x:removePunct(x))
df.head()

,category,headline,authors,link,short_description,date,text,textClean
0,WORLD NEWS,South Korean President Meets North Korea's Kim...,,https://www.huffingtonpost.com/entry/south-kor...,The two met to pave the way for a summit betwe...,2018-05-26,South Korean President Meets North Korea's Kim...,South Korean President Meets North Koreas Kim ...
1,WORLD NEWS,North Korea Still Open To Talks After Trump Ca...,"Josh Smith and Christine Kim, Reuters",https://www.huffingtonpost.com/entry/north-kor...,Trump’s announcement came after repeated threa...,2018-05-25,North Korea Still Open To Talks After Trump Ca...,North Korea Still Open To Talks After Trump Ca...
2,WORLD NEWS,2 Men Detonate Bomb Inside Indian Restaurant N...,,https://www.huffingtonpost.com/entry/mississau...,"Fifteen people were taken to the hospital, thr...",2018-05-25,2 Men Detonate Bomb Inside Indian Restaurant N...,2 Men Detonate Bomb Inside Indian Restaurant N...
3,WORLD NEWS,Thousands Travel Home To Ireland To Vote On Ab...,Antonia Blumberg,https://www.huffingtonpost.com/entry/irish-tra...,Just try to read these #HomeToVote tweets with...,2018-05-25,Thousands Travel Home To Ireland To Vote On Ab...,Thousands Travel Home To Ireland To Vote On Ab...
4,WORLD NEWS,Irish Voters Set To Liberalize Abortion Laws I...,,https://www.huffingtonpost.com/entry/ireland-a...,Vote counting will begin Saturday.,2018-05-25,Irish Voters Set To Liberalize Abortion Laws I...,Irish Voters Set To Liberalize Abortion Laws I...


In [ ]:
import re

In [ ]:
def tokenize(txt):
  tokens=re.split('\W+',txt)
  return tokens

In [ ]:
df['textCleanToken']=df['textClean'].apply(lambda x:tokenize(x.lower()))
df.head()

,category,headline,authors,link,short_description,date,text,textClean,textCleanToken
0,WORLD NEWS,South Korean President Meets North Korea's Kim...,,https://www.huffingtonpost.com/entry/south-kor...,The two met to pave the way for a summit betwe...,2018-05-26,South Korean President Meets North Korea's Kim...,South Korean President Meets North Koreas Kim ...,"[south, korean, president, meets, north, korea..."
1,WORLD NEWS,North Korea Still Open To Talks After Trump Ca...,"Josh Smith and Christine Kim, Reuters",https://www.huffingtonpost.com/entry/north-kor...,Trump’s announcement came after repeated threa...,2018-05-25,North Korea Still Open To Talks After Trump Ca...,North Korea Still Open To Talks After Trump Ca...,"[north, korea, still, open, to, talks, after, ..."
2,WORLD NEWS,2 Men Detonate Bomb Inside Indian Restaurant N...,,https://www.huffingtonpost.com/entry/mississau...,"Fifteen people were taken to the hospital, thr...",2018-05-25,2 Men Detonate Bomb Inside Indian Restaurant N...,2 Men Detonate Bomb Inside Indian Restaurant N...,"[2, men, detonate, bomb, inside, indian, resta..."
3,WORLD NEWS,Thousands Travel Home To Ireland To Vote On Ab...,Antonia Blumberg,https://www.huffingtonpost.com/entry/irish-tra...,Just try to read these #HomeToVote tweets with...,2018-05-25,Thousands Travel Home To Ireland To Vote On Ab...,Thousands Travel Home To Ireland To Vote On Ab...,"[thousands, travel, home, to, ireland, to, vot..."
4,WORLD NEWS,Irish Voters Set To Liberalize Abortion Laws I...,,https://www.huffingtonpost.com/entry/ireland-a...,Vote counting will begin Saturday.,2018-05-25,Irish Voters Set To Liberalize Abortion Laws I...,Irish Voters Set To Liberalize Abortion Laws I...,"[irish, voters, set, to, liberalize, abortion,..."


In [ ]:
import nltk
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def removeStopwords(txt):
  txt_clean=[word for word in txt if word not in stopwords]
  return txt_clean

In [ ]:
df['noStopwords']=df['textCleanToken'].apply(lambda x: removeStopwords(x))
df.head()

,category,headline,authors,link,short_description,date,text,textClean,textCleanToken,noStopwords
0,WORLD NEWS,South Korean President Meets North Korea's Kim...,,https://www.huffingtonpost.com/entry/south-kor...,The two met to pave the way for a summit betwe...,2018-05-26,South Korean President Meets North Korea's Kim...,South Korean President Meets North Koreas Kim ...,"[south, korean, president, meets, north, korea...","[south, korean, president, meets, north, korea..."
1,WORLD NEWS,North Korea Still Open To Talks After Trump Ca...,"Josh Smith and Christine Kim, Reuters",https://www.huffingtonpost.com/entry/north-kor...,Trump’s announcement came after repeated threa...,2018-05-25,North Korea Still Open To Talks After Trump Ca...,North Korea Still Open To Talks After Trump Ca...,"[north, korea, still, open, to, talks, after, ...","[north, korea, still, open, talks, trump, canc..."
2,WORLD NEWS,2 Men Detonate Bomb Inside Indian Restaurant N...,,https://www.huffingtonpost.com/entry/mississau...,"Fifteen people were taken to the hospital, thr...",2018-05-25,2 Men Detonate Bomb Inside Indian Restaurant N...,2 Men Detonate Bomb Inside Indian Restaurant N...,"[2, men, detonate, bomb, inside, indian, resta...","[2, men, detonate, bomb, inside, indian, resta..."
3,WORLD NEWS,Thousands Travel Home To Ireland To Vote On Ab...,Antonia Blumberg,https://www.huffingtonpost.com/entry/irish-tra...,Just try to read these #HomeToVote tweets with...,2018-05-25,Thousands Travel Home To Ireland To Vote On Ab...,Thousands Travel Home To Ireland To Vote On Ab...,"[thousands, travel, home, to, ireland, to, vot...","[thousands, travel, home, ireland, vote, abort..."
4,WORLD NEWS,Irish Voters Set To Liberalize Abortion Laws I...,,https://www.huffingtonpost.com/entry/ireland-a...,Vote counting will begin Saturday.,2018-05-25,Irish Voters Set To Liberalize Abortion Laws I...,Irish Voters Set To Liberalize Abortion Laws I...,"[irish, voters, set, to, liberalize, abortion,...","[irish, voters, set, liberalize, abortion, law..."


In [ ]:
#from nltk.stem import PorterStemmer
#ps=PorterStemmer()
#def stemming(token_txt):
#  text=[ps.stem(word) for word in token_txt]
#  return text

In [ ]:
#df['stemmed']=df['noStopwords'].apply(lambda x: stemming(x))
#df.head()

In [ ]:
def cleanWords(text):
  txt="".join([p for p in text if p not in string.punctuation])
  tokens=re.split("\W+",txt)
  txt=[ps.stem(word) for word in tokens if word not in stopwords]

In [ ]:
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [ ]:
df['string']=df['noStopwords'].apply(lambda x: listToString(x))
df.head()

,category,headline,authors,link,short_description,date,text,textClean,textCleanToken,noStopwords,string
0,WORLD NEWS,South Korean President Meets North Korea's Kim...,,https://www.huffingtonpost.com/entry/south-kor...,The two met to pave the way for a summit betwe...,2018-05-26,South Korean President Meets North Korea's Kim...,South Korean President Meets North Koreas Kim ...,"[south, korean, president, meets, north, korea...","[south, korean, president, meets, north, korea...",south korean president meets north koreas kim ...
1,WORLD NEWS,North Korea Still Open To Talks After Trump Ca...,"Josh Smith and Christine Kim, Reuters",https://www.huffingtonpost.com/entry/north-kor...,Trump’s announcement came after repeated threa...,2018-05-25,North Korea Still Open To Talks After Trump Ca...,North Korea Still Open To Talks After Trump Ca...,"[north, korea, still, open, to, talks, after, ...","[north, korea, still, open, talks, trump, canc...",north korea still open talks trump cancels sum...
2,WORLD NEWS,2 Men Detonate Bomb Inside Indian Restaurant N...,,https://www.huffingtonpost.com/entry/mississau...,"Fifteen people were taken to the hospital, thr...",2018-05-25,2 Men Detonate Bomb Inside Indian Restaurant N...,2 Men Detonate Bomb Inside Indian Restaurant N...,"[2, men, detonate, bomb, inside, indian, resta...","[2, men, detonate, bomb, inside, indian, resta...",2 men detonate bomb inside indian restaurant n...
3,WORLD NEWS,Thousands Travel Home To Ireland To Vote On Ab...,Antonia Blumberg,https://www.huffingtonpost.com/entry/irish-tra...,Just try to read these #HomeToVote tweets with...,2018-05-25,Thousands Travel Home To Ireland To Vote On Ab...,Thousands Travel Home To Ireland To Vote On Ab...,"[thousands, travel, home, to, ireland, to, vot...","[thousands, travel, home, ireland, vote, abort...",thousands travel home ireland vote abortion ac...
4,WORLD NEWS,Irish Voters Set To Liberalize Abortion Laws I...,,https://www.huffingtonpost.com/entry/ireland-a...,Vote counting will begin Saturday.,2018-05-25,Irish Voters Set To Liberalize Abortion Laws I...,Irish Voters Set To Liberalize Abortion Laws I...,"[irish, voters, set, to, liberalize, abortion,...","[irish, voters, set, liberalize, abortion, law...",irish voters set liberalize abortion laws land...


In [ ]:
def removeNum(txt):
  result = ''.join([i for i in txt if not i.isdigit()])
  return result

In [ ]:
df['Nonum']=df['string'].apply(lambda x: removeNum(x))

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df

,category,headline,authors,link,short_description,date,text,textClean,textCleanToken,noStopwords,string,Nonum
0,SPORTS,Why Dave Brandon Won't Be Michigan's Athletic ...,"Justin Knapp, ContributorStrategy, Personal & ...",https://www.huffingtonpost.com/entry/why-dave-...,The meltdown at Michigan following back-to-bac...,2014-10-18,Why Dave Brandon Won't Be Michigan's Athletic ...,Why Dave Brandon Wont Be Michigans Athletic Di...,"[why, dave, brandon, wont, be, michigans, athl...","[dave, brandon, wont, michigans, athletic, dir...",dave brandon wont michigans athletic director ...,dave brandon wont michigans athletic director ...
1,WOMEN,Ciara Was Slut-Shamed For Wearing A Floor-Leng...,Zeba Blay,https://www.huffingtonpost.com/entry/ciara-was...,"Calm down, people. They're just boobs.",2016-01-12,Ciara Was Slut-Shamed For Wearing A Floor-Leng...,Ciara Was SlutShamed For Wearing A FloorLength...,"[ciara, was, slutshamed, for, wearing, a, floo...","[ciara, slutshamed, wearing, floorlength, gown...",ciara slutshamed wearing floorlength gown cape...,ciara slutshamed wearing floorlength gown cape...
2,SCIENCE,Adorable Polar Bear Cub Nora Grows Up Before Y...,Lee Moran,https://www.huffingtonpost.com/entry/polar-bea...,Cuteness alert!,2016-02-06,Adorable Polar Bear Cub Nora Grows Up Before Y...,Adorable Polar Bear Cub Nora Grows Up Before Y...,"[adorable, polar, bear, cub, nora, grows, up, ...","[adorable, polar, bear, cub, nora, grows, eyes...",adorable polar bear cub nora grows eyescutenes...,adorable polar bear cub nora grows eyescutenes...
3,WOMEN,The Rape Victims Silenced By Their Prison Cells,Melissa Jeltsen,https://www.huffingtonpost.com/entry/sexual-as...,A prison chaplain who admitted to sexually ass...,2016-07-16,The Rape Victims Silenced By Their Prison Cell...,The Rape Victims Silenced By Their Prison Cell...,"[the, rape, victims, silenced, by, their, pris...","[rape, victims, silenced, prison, cellsa, pris...",rape victims silenced prison cellsa prison cha...,rape victims silenced prison cellsa prison cha...
4,SPORTS,"LeBron James Starts Game Without Headband, Pos...",Lucy McCalmont,https://www.huffingtonpost.com/entry/lebron-ja...,It's a good look for him.,2015-03-11,"LeBron James Starts Game Without Headband, Pos...",LeBron James Starts Game Without Headband Poss...,"[lebron, james, starts, game, without, headban...","[lebron, james, starts, game, without, headban...",lebron james starts game without headband poss...,lebron james starts game without headband poss...
...,...,...,...,...,...,...,...,...,...,...,...,...
14431,SPORTS,Prosecutor Refutes Patrick Kane Rape Kit Tampe...,"Carolyn Thompson, AP",https://www.huffingtonpost.com/entry/patrick-k...,Erie County District Attorney Frank Sedita cal...,2015-09-25,Prosecutor Refutes Patrick Kane Rape Kit Tampe...,Prosecutor Refutes Patrick Kane Rape Kit Tampe...,"[prosecutor, refutes, patrick, kane, rape, kit...","[prosecutor, refutes, patrick, kane, rape, kit...",prosecutor refutes patrick kane rape kit tampe...,prosecutor refutes patrick kane rape kit tampe...
14432,WOMEN,Love?,"Lacey Johnson, ContributorWriter, Journalist &...",https://www.huffingtonpost.com/entry/love_5_b_...,"One may label it 'loving' all day long, profes...",2015-07-07,"Love?One may label it 'loving' all day long, p...",LoveOne may label it loving all day long profe...,"[loveone, may, label, it, loving, all, day, lo...","[loveone, may, label, loving, day, long, profe...",loveone may label loving day long profess roof...,loveone may label loving day long profess roof...
14433,SCIENCE,"NASA's Best Space Images Of The Week, Jan. 27 ...",Melissa Cronin,https://www.huffingtonpost.com/entry/best-spac...,NASA is well-known for producing some of the m...,2013-01-27,"NASA's Best Space Images Of The Week, Jan. 27 ...",NASAs Best Space Images Of The Week Jan 27 PHO...,"[nasas, best, space, images, of, the, week, ja...","[nasas, best, space, images, week, jan, 27, ph...",nasas best space images week jan 27 photosn

##DOC2VEC

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
import nltk


nltk.download('punkt')
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(df['Nonum'])]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tagged_data

[TaggedDocument(words=['dave', 'brandon', 'wont', 'michigans', 'athletic', 'director', 'next', 'yearthe', 'meltdown', 'michigan', 'following', 'backtoback', 'home', 'losses', 'severe', 'mishandling', 'concussed', 'quarterback', 'shane', 'morris', 'left', 'realm', 'passion', 'entered', 'world', 'profession', 'may', 'love', 'football', 'career', 'communications'], tags=['0']),
 TaggedDocument(words=['ciara', 'slutshamed', 'wearing', 'floorlength', 'gown', 'capecalm', 'people', 'theyre', 'boobs'], tags=['1']),
 TaggedDocument(words=['adorable', 'polar', 'bear', 'cub', 'nora', 'grows', 'eyescuteness', 'alert'], tags=['2']),
 TaggedDocument(words=['rape', 'victims', 'silenced', 'prison', 'cellsa', 'prison', 'chaplain', 'admitted', 'sexually', 'assaulting', 'three', 'inmates', 'may', 'spend', 'less', 'year', 'behind', 'bars', 'dont', 'seem', 'care'], tags=['3']),
 TaggedDocument(words=['lebron', 'james', 'starts', 'game', 'without', 'headband', 'possibly', 'first', 'time', 'everits', 'good',

In [ ]:
max_epochs = 100
vec_size = 75
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [ ]:
nnvec=model.docvecs.vectors_docs

In [ ]:
nnvecdf = pd.DataFrame(nnvec)

In [ ]:
y=df['category']

In [ ]:
import tensorflow as tf
x=nnvecdf.values

##LE and TRAIN/TEST SPLIT

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

yLE = le.fit_transform(y)

In [ ]:
yLE = np.asarray( tf.keras.utils.to_categorical(yLE))

In [ ]:
yLE

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainy, testy = train_test_split(x, yLE, test_size=0.25, random_state=1000)

In [ ]:
trainX

array([[ 0.05274644,  0.78456897,  0.19252352, ..., -1.0852604 ,
         0.6434279 , -0.87761796],
       [-0.9551712 , -1.4646782 , -1.8022014 , ..., -4.0328298 ,
         0.8007793 , -1.9220097 ],
       [-0.17499295,  2.0972795 ,  3.3799858 , ...,  0.36732063,
        -0.7330734 , -2.2033114 ],
       ...,
       [ 0.9690667 , -0.21622917,  0.66816473, ...,  0.60618967,
        -0.4857822 ,  1.4745469 ],
       [-2.1144934 ,  1.114842  ,  2.6769507 , ..., -4.1568046 ,
        -2.083606  ,  0.17419562],
       [ 0.9091223 , -0.92557365, -0.3139882 , ..., -2.3563242 ,
        -0.65265137, -0.5315482 ]], dtype=float32)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
import pickle

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# transform data
trainX = scaler.fit_transform(trainX)

In [ ]:
testX = scaler.fit_transform(testX)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam

In [ ]:
model=Sequential()

model.add(Embedding(2,64,input_length=vec_size))
model.add(LSTM(128,dropout=0.1))
model.add(Dense(5,activation='sigmoid'))
optimizer=Adam(learning_rate=3e-4)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
history=model.fit(trainX, trainy, epochs=20, validation_data=(testX,testy))

Epoch 1/20
339/339 [==============================] - 54s 103ms/step - loss: 1.5489 - accuracy: 0.3343 - val_loss: 1.5437 - val_accuracy: 0.3261
Epoch 2/20
339/339 [==============================] - 34s 101ms/step - loss: 1.5291 - accuracy: 0.3408 - val_loss: 1.5449 - val_accuracy: 0.3261
Epoch 3/20
339/339 [==============================] - 34s 99ms/step - loss: 1.5350 - accuracy: 0.3387 - val_loss: 1.5445 - val_accuracy: 0.3261
Epoch 4/20
339/339 [==============================] - 34s 101ms/step - loss: 1.5357 - accuracy: 0.3378 - val_loss: 1.5429 - val_accuracy: 0.3261
Epoch 5/20
339/339 [==============================] - 33s 99ms/step - loss: 1.5246 - accuracy: 0.3484 - val_loss: 1.5439 - val_accuracy: 0.3261
Epoch 6/20
339/339 [==============================] - 33s 96ms/step - loss: 1.5246 - accuracy: 0.3513 - val_loss: 1.5437 - val_accuracy: 0.3261
Epoch 7/20
339/339 [==============================] - 33s 98ms/step - loss: 1.5296 - accuracy: 0.3495 - val_loss: 1.5433 - val_accura